In [ ]:
import json
filename = '../data/results2.json'

with open(filename) as f:
    data = f.readlines()

data = eval(data[0])

In [ ]:
with open('../data/results.json') as f: 
    print(json.load(f))

In [ ]:
data[0].keys()

In [ ]:
data[0]['orf']

In [ ]:
data